In [1]:
import json
import requests
import sys
from datetime import datetime, time

sys.path.insert(0, '../..')


from virasana.integracao.due import raspa_due

VIRASANA_URL = "http://localhost/virasana/"
VIRASANA_URL = "http://10.68.64.12/virasana/"

2019-09-10 15:30:47,076 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../..\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../..\ajna_commons\flask\access.log


## Realizando consulta no MongoDB Gridfs do AJNA via API virasana

In [43]:
diaapesquisar = datetime.today()
diaapesquisar = datetime(2019, 9, 2)

In [44]:
datainicial = datetime.strftime(datetime.combine(diaapesquisar, time.min), '%Y-%m-%d  %H:%M:%S')
datafinal = datetime.strftime(datetime.combine(diaapesquisar, time.max), '%Y-%m-%d %H:%M:%S')
print(datainicial, datafinal)

2019-09-02  00:00:00 2019-09-02 23:59:59


In [49]:
params = {'query': 
          { 'metadata.dataescaneamento': {'$gte': datainicial, '$lte': datafinal},
            'metadata.contentType': 'image/jpeg',
            'metadata.carga.manifesto.tipomanifesto': 'lce'
          },
          'projection': 
          {'metadata.numeroinformado': 1,
           'metadata.dataescaneamento': 1}
         }

r = requests.post(VIRASANA_URL + "grid_data", json=params)
lista_lce = list(r.json())
print(len(lista_lce))

params['query']['metadata.carga.manifesto.tipomanifesto'] = None
r = requests.post(VIRASANA_URL + "grid_data", json=params)
lista_null = list(r.json())
print(len(lista_null))

# print(r.url)
# print(r.text)

428
504


In [50]:
# lista = [['MSCU6656780', '']]
lista = [*lista_lce, *lista_null]
conteineres_ids = {linha['metadata']['numeroinformado']: linha['_id'] for linha in lista}
conteineres = list(conteineres_ids.keys())

# print(conteineres_ids)


## Conectar na API do Pucomex/suiterfb para encontrar e baixar DUE 

In [51]:
%%time
from selenium import webdriver

GECKO_PATH = 'C:\\Users\\25052288840\\Downloads\\chromedriver.exe'
driver = webdriver.Chrome(GECKO_PATH)
raspa_due.auth_suite_rfb(driver)
try:
    %time conteineres_listadue = raspa_due.get_dues_pos_acd(driver, conteineres)
    # print(conteineres_listadue)
    %time due_detalhe = raspa_due.detalha_dues(driver, conteineres_listadue)
    # print(conteineres_due)
finally:
    driver.close()

    

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Wall time: 24.9 s
Wall time: 6min 27s


## Conectar na API do virasana para fazer UPLOAD do arquivo JSON resultante

O arquivo seguirá o formato [{_id: {Documento DUE}}]

In [52]:
print(len(conteineres_listadue))
print(len(due_detalhe))

433
17


In [53]:
for conteiner, dues in list(conteineres_listadue.items())[:40]:
    if dues is not None and len(dues) > 0:
        print(conteiner, dues)
        
for due, conteudo in list(due_detalhe.items())[:10]:
    print(due, json.dumps(conteudo)[:30])

HLXU3244445 ['19BR0011834993']
EITU1742327 ['19BR0012120340']
CXRU1531943 ['19BR0011677170']
MSCU6935403 ['19BR0011573502']
BEAU4408111 ['19BR0012120340']
SEGU5312422 ['19BR0012120340']
CPSU1714556 ['19BR0011834993']
TCLU6305812 ['19BR0012120340']
TLLU4000598 ['19BR0012120340']
FSCU4974722 ['19BR0012091226']
19BR0011834993 {"canal": "VERDE", "canalBD": 
19BR0012120340 {"canal": "VERDE", "canalBD": 
19BR0011677170 {"canal": "LARANJA", "canalBD"
19BR0011573502 {"canal": "VERDE", "canalBD": 
19BR0012091226 {"canal": "VERDE", "canalBD": 
19BR0011425902 {"canal": "VERDE", "canalBD": 
19BR0011681437 {"canal": "VERDE", "canalBD": 
19BR0011489765 {"canal": "VERDE", "canalBD": 
19BR0011809034 {"canal": "VERDE", "canalBD": 
19BR0011816375 {"canal": "VERDE", "canalBD": 


In [54]:
with open('dues.txt', 'w') as out:
    json.dump(due_detalhe, out)

In [55]:
with open('dues.txt', 'r') as due_in:
    dues_ = json.load(due_in)

In [37]:
dues_['19BR0012120340']

{'canal': 'VERDE',
 'canalBD': 1,
 'chaveAcesso': '19HZO022607642',
 'dataProcessamentoTa': '2019-09-05T20:06:19.000+0000',
 'exportadorEstrangeiro': False,
 'declarantePJBD': '07401436000131',
 'descricaoResultadoProcessarTA': 'Dispensado',
 'despachoEmRecintoAlfandegado': True,
 'embarqueEmRecintoAlfandegado': True,
 'despachoEmRecintoDomiciliar': False,
 'formaExportacao': 'Por conta própria',
 'formaExportacaoBD': 1,
 'id': 2292020,
 'indicadorDAT': False,
 'indicadorExigencia': False,
 'indicadorImpedeEmbarqueTA': 'N',
 'indicadorInspecao': 'N',
 'indicadorOEA': False,
 'informacoesComplementares': 'RM 81147404',
 'usuarioConfaz': False,
 'consultaConfaz': False,
 'listaExigencias': [],
 'listaHistorico': [{'dataHoraEvento': '05/09/2019 - 17:07:16',
   'evento': 'Registro',
   'informacoesAdicionais': 'Origem: Serviço',
   'responsavel': '22536715841',
   'tipoEvento': 'REGISTRADA'},
  {'dataHoraEvento': '05/09/2019 - 17:07:19',
   'evento': 'Apresentação para despacho',
   'respo

In [56]:
def monta_due_ajna(due):
    def get_dados_recinto(recinto_dict):
        result = {}
        result['codigo'] = recinto_dict.get('codigo')
        depositario = recinto_dict.get('depositario')
        if depositario:
            result['depositario'] = depositario.get('depositario')
            result['nome'] = depositario.get('nome')
            result['descricao'] = depositario.get('descricao')
            unidade = depositario.get('unidadeLocalRFB')
            if unidade:
                unidadeLocalRFB = depositario.get('codigo')

        return result 
    
    keys = ['canal', 'chaveAcesso', 'dataProcessamentoTa', 'descricaoTipoItemDue',
            'exportadorEstrangeiro', 'formaExportacao', 'indicadorOEA', 'informacoesComplementares']
    pacote = {}
    for key in keys:
        item = due.get(key)
        if item is not None:
            pacote[key] = item

    declarante = due.get('niDeclarante')
    if declarante:
        pacote['Declarante'] = declarante.get('numero')
        pacote['Nome Declarante'] = declarante.get('nome')

    destino = due.get('paisImportador')
    if destino:
        pacote['PaisImportador'] = destino.get('nome')

    lista_items = due.get('listaInfoItemDue')
    itensDue = []
    for item in lista_items:
        itemDue = {}
        itemDue['descricaoMercadoria'] = item.get('descricaoMercadoria')
        itemDue['exportadorEstrangeiro'] = item.get('exportadorEstrangeiro')
        ncm = item.get('ncm')
        if ncm:
            itemDue['ncm'] = ncm.get('codigo')
        exportador = item.get('niExportador')
        if exportador:
            itemDue['Exportador'] = exportador.get('numero')
            itemDue['NomeExportador'] = exportador.get('nome')
        itensDue.append(itemDue)
    pacote['itens'] = itensDue
    
    for recinto_tipo in ['recintoAduaneiroDespacho',  'recintoAduaneiroEmbarque']:
        pacote[recinto_tipo] = get_dados_recinto(due.get(recinto_tipo))
    ruc = due.get('ruc')
    if ruc:
        pacote['ruc'] = ruc.get('numero')
    pacote['listaHistorico'] = due.get('listaHistorico')

    return pacote

    


In [39]:
pacote = monta_due_ajna(dues_['19BR0012120340'])
pacote

{'canal': 'VERDE',
 'chaveAcesso': '19HZO022607642',
 'dataProcessamentoTa': '2019-09-05T20:06:19.000+0000',
 'descricaoTipoItemDue': ' Nota Fiscal Eletrônica (NF-e)',
 'exportadorEstrangeiro': False,
 'formaExportacao': 'Por conta própria',
 'indicadorOEA': False,
 'informacoesComplementares': 'RM 81147404',
 'Declarante': '07401436000131',
 'Nome Declarante': 'ELDORADO BRASIL CELULOSE S/A',
 'PaisImportador': 'AUSTRIA',
 'itens': [{'descricaoMercadoria': 'Celulose Kraft Fardo 02 Arames CW',
   'exportadorEstrangeiro': False,
   'ncm': '47032900',
   'Exportador': '07401436000131',
   'NomeExportador': 'ELDORADO BRASIL CELULOSE S/A'}],
 'recintoAduaneiroDespacho': {'codigo': '8931404',
  'depositario': None,
  'nome': 'EMBRAPORT EMPRESA BRASILEIRA DE TERMINAIS PORTUARIOS S/A',
  'descricao': None},
 'recintoAduaneiroEmbarque': {'codigo': '8931404',
  'depositario': None,
  'nome': 'EMBRAPORT EMPRESA BRASILEIRA DE TERMINAIS PORTUARIOS S/A',
  'descricao': None},
 'ruc': '9BR07401436136

In [57]:
pacote_carregamento = {}
for conteiner, numeros_dues in conteineres_listadue.items():
    _id = conteineres_ids[conteiner]
    lista_dues = []
    for due in numeros_dues:
        if due is not None:
            pacote = monta_due_ajna(due_detalhe[due])
            lista_dues.append({'numero': due, **pacote})
    if numeros_dues and len(numeros_dues) > 0:
        pacote_carregamento[_id] = lista_dues
    

KeyError: 'MRKU0751849'

In [41]:
pacote_carregamento

{'5d6fc07505d4bedc177c0611': [{'numero': '19BR0011834993',
   'canal': 'VERDE',
   'chaveAcesso': '19RUK022324101',
   'dataProcessamentoTa': '2019-09-02T09:00:13.000+0000',
   'descricaoTipoItemDue': ' Nota Fiscal Eletrônica (NF-e)',
   'exportadorEstrangeiro': False,
   'formaExportacao': 'Por conta própria',
   'indicadorOEA': False,
   'Declarante': '33729690001811',
   'Nome Declarante': 'ED&amp;F MAN VOLCAFE BRASIL LTDA',
   'PaisImportador': 'SUICA',
   'itens': [{'descricaoMercadoria': 'CAFE DO BRASIL CRU EM GRAO NAO DESCAFEINADO ARABICA NY 2/3 PENEIRAS 09/11 SAFRA 2018/2019',
     'exportadorEstrangeiro': False,
     'ncm': '9011110',
     'Exportador': '33729690001811',
     'NomeExportador': 'ED&amp;F MAN VOLCAFE BRASIL LTDA'},
    {'descricaoMercadoria': 'CAFE DO BRASIL CRU EM GRAO NAO DESCAFEINADO ARABICA NY 2/3 PENEIRAS 09/11 SAFRA 2018/2019',
     'exportadorEstrangeiro': False,
     'ncm': '9011110',
     'Exportador': '33729690001811',
     'NomeExportador': 'ED&amp;F 

In [ ]:
r = requests.post(VIRASANA_URL + "dues/update", json=pacote_carregamento)
print(r.status_code)
print(r.text)
